In [1]:

from termcolor import colored
from IPython.display import Markdown, display

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_community.vectorstores import Qdrant, Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()



True

In [2]:
openai_key = os.getenv("OPENAI_API_KEY")

In [3]:
def printmd(string):
    display(Markdown(string))

# RAG

## Loading web news data to the database

In [4]:
urls = ["https://pt.wikipedia.org/wiki/Javier_Milei",
        "https://www.cbsnews.com/news/super-bowl-winners-list-history/",
        "https://ge.globo.com/futebol/times/corinthians/noticia/2024/02/16/corinthians-anuncia-a-contratacao-do-lateral-direito-matheuzinho.ghtml",
        "https://ge.globo.com/futebol/times/botafogo/noticia/2024/02/16/luiz-henrique-do-botafogo-tem-lesao-na-panturrilha.ghtml",
        "https://ge.globo.com/futebol/times/fluminense/noticia/2024/02/16/escalacao-do-fluminense-diniz-escala-reservas-contra-madureira-mas-tera-john-kennedy-e-douglas-costa.ghtml",
    "https://www.lemonde.fr/football/article/2024/02/25/ligue-1-le-psg-arrache-le-nul-de-justesse-contre-rennes-monaco-remonte-sur-le-podium_6218528_1616938.html",
    "https://ge.globo.com/futebol/futebol-internacional/futebol-frances/copa-da-franca/noticia/2024/02/27/perri-e-heroi-nos-penaltis-e-lyon-avanca-de-fase-na-copa-da-franca.ghtml",
    "https://ge.globo.com/tenis/noticia/2024/02/27/joao-fonseca-sofre-com-condicoes-ruins-da-quadra-no-atp-santiago-e-e-eliminado.ghtml",
    
]
loader = SeleniumURLLoader(urls=urls)
docs = loader.load()

## Loading text from PDF using PyPDFLoader

In [5]:
docs

[Document(page_content='Alternar o índice\n\nConteúdo\n\nInício\n\n1Primeiros anos\n\n2Carreira econômica\n\n3Carreira política\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\tAlternar a subsecção Carreira política\n\t\t\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t3.1Candidatura à presidência (2023)\n\n4Presidência (2023–presente)\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\tAlternar a subsecção Presidência (2023–presente)\n\t\t\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t4.1Primeiros atos\n\n5Posições políticas\n\n6Críticas\n\n7Rádio\n\n8Livros\n\n9Artigos acadêmicos selecionados\n\n10Ver também\n\n11Referências\n\n12Ligações externas\n\nJavier Milei\n\n68 línguas\n\nAfrikaans\n\nالعربية\n\nمصرى\n\nAzərbaycanca\n\nБеларуская\n\nБългарски\n\nBrezhoneg\n\nCatalà\n\nکوردی\n\nČeština\n\nCymraeg\n\nDeutsch\n\nΕλληνικά\n\nEnglish\n\nEsperanto\n\nEspañol\n\nEesti\n\nEuskara\n\nفارسی\n\nSuomi\n\nFrançais\n\nGaeilge\n\nGalego\n\nAvañe\'ẽ\n\nעברית\n\nHrvatski\n\nMagyar\n\nՀայերեն\n\nBahasa Indonesia

In [7]:
# Splitting the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
splits = text_splitter.split_documents(docs)

In [16]:
# load from disk
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory="./chroma_db")
vectorstore


In [17]:
# RAG Chain
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# Multiple News Queries

In [18]:
query = ["Quem é o atual presidente da Argentina?",
         "Quem ganhou o Super Bowl 2024?",
         "Quem é o jogador novo do Corinthians?",
         "Quem é o jogador Botafoguense que se contundiu recenteente?",
         "Qual a formação usada no Fluminense no jogo contra o Madureira?",
         "Quem é o novo reforço do Fluminense?",
         "Qual a nova resulução sobre 'saidinhas' de presos?",
         ]
for idx, q in enumerate(query):
    print(f"Query {idx+1}: {q}")
    print(f"    RAG-Answer > {rag_chain_with_source.invoke(q)}\n")
    print(f"    GPT-Answer > {llm.invoke(q).content}\n")
    

Query 1: Quem é o atual presidente da Argentina?
    RAG-Answer > {'context': [Document(page_content='Wikiquote\n\nOrigem: Wikipédia, a enciclopédia livre.\n\nKarina Milei.\n\nJavier Milei Javier Milei Foto oficial, 2023. 52.° Presidente da Argentina Período 10 de dezembro de 2023 até a atualidade Vice-presidente Victoria Villarruel Antecessor(a) Alberto Fernández Deputado da Nação Argentina por Buenos Aires Período 10 de dezembro de 2021 até 29 de novembro de 2023 Dados pessoais Nome completo Javier Gerardo Milei Nascimento 22 de outubro de 1970 (53\xa0anos) Buenos Aires , Argentina Nacionalidade argentino Alma mater Universidade de Belgrano Instituto de Desarrollo Económico y Social Universidad Torcuato Di Tella Partido A Liberdade Avança (2021–presente) Avança Liberdade (2020–2021) Religião Catolicismo [ 1 ] Profissão Presidente da Argentina , Economista , escritor , professor Assinatura', metadata={'description': 'No description found.', 'language': 'pt', 'source': 'https://pt.wiki